In [1]:
import sys

!which {sys.executable}

/ut3/jerome/miniconda3/envs/wakai/bin/python


In [3]:
import sys
sys.path.insert(0, '../code')

# TOI catalog

In [4]:
from catalog import get_tois

tois = get_tois()
# tois.columns.tolist()

In [5]:
tois[tois.Comments.apply(lambda x: 'young' in str(x))]

,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Metallicity err,Stellar Mass (M_Sun),Stellar Mass (M_Sun) err,Sectors,Date TOI Alerted (UTC),Date TOI Updated (UTC),Date Modified,Comments,ra_deg,dec_deg
732,146520535,942.01,NaN,3,5,5,3,4,4,4,...,NaN,0.814000,0.097250,5,2019-07-24,2020-05-20,2022-12-14 12:09:24,Probable young K dwarf. Transit is believable ...,76.649625,-20.245614
2902,330135981,3604.01,NaN,3,4,4,3,2,4,4,...,NaN,NaN,NaN,"16,17,56,57",2021-06-23,2023-02-01,2023-04-17 12:02:58,V-shaped; found in faint-star QLP search; no s...,333.501042,56.918981
4902,308300731,5743.01,NaN,3,4,3,3,4,4,4,...,NaN,0.898000,0.109025,"26,53,79,80",2022-09-01,2024-08-25,2024-08-26 00:00:00,radius possibly underestimated; potentially yo...,267.904333,24.427456
5099,236785891,5956.01,TIC 236785891.01,3,4,3,3,2,4,4,...,NaN,0.483975,0.021842,"56,57",2022-11-16,2023-02-03,2023-08-10 00:00:00,V-shaped; large; young host star; ~5 sigma odd...,283.317250,59.817658
5389,435878153,6258.01,NaN,2,4,4,2,4,4,4,...,NaN,0.368918,0.020209,"17,57",2023-04-06,2023-04-06,2024-02-23 00:00:00,low SNR; asymmetric transit shape; flaring pos...,13.085375,20.938875
5806,253864764,6678.01,NaN,1,4,1,1,1,4,4,...,NaN,0.770000,0.092773,67,2023-09-21,2023-09-21,2023-09-25 12:14:10,rotational modulation at 7-8 days may indicate...,283.536292,-36.656858


In [6]:
tois.shape

(6160, 64)

# cross-match TOI with Gaia DR3 using `astroquery.xmatch`

In [7]:
import astropy.units as u
from astroquery.xmatch import XMatch
import pandas as pd

gaia_table = XMatch.query(
    cat1=open('../data/TOIs.csv'),
    cat2="vizier:I/355/gaiadr3",  # The Vizier identifier for Gaia DR3
    max_distance=1 * u.arcsec,    # Set the maximum matching radius
    colRA1="ra_deg",  # Column name for Right Ascension in TIC results
    colDec1="dec_deg" # Column name for Declination in TIC results
)
gaia_df = gaia_table.to_pandas()

# Apply magnitude filtering criteria
# For main-sequence stars, the difference between TESS T and Gaia G is around 0–0.5 magnitudes
# Redder stars might have T - G values closer to 1–2 magnitudes.
# For giants or highly reddened stars, the difference could be larger.
if True:
    magnitude_threshold = 1
    matches = gaia_df[abs(gaia_df['Gmag'] - gaia_df['TESS Mag']) < magnitude_threshold]
if False:
    teff_threshold = 1000
    matches = gaia_df[abs(gaia_df['Teff'] - gaia_df['Stellar Eff Temp (K)']) < teff_threshold]
matches.head()

Could not import regions, which is required for some of the functionalities of this module.


,angDist,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,...,B_AG,E(BP-RP),b_E(BP-RP),B_E(BP-RP),Lib,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,RADEcorJ2000
0,0.045188,164249928,6797.01,NaN,3,4,3,3,1,4,...,0.3607,0.1857,0.1729,0.1953,MARCS,304.395077,-28.821273,0.706557,0.558226,0.4318
1,0.030103,290403522,6798.01,TIC 290403522.01,3,4,4,3,1,4,...,0.0154,0.0063,0.0041,0.0083,PHOENIX,330.957845,-72.440757,0.254609,0.271682,-0.2895
2,0.024988,387295547,6799.01,NaN,3,4,3,3,1,4,...,0.0228,0.0101,0.0080,0.0124,MARCS,312.696456,-58.943253,0.163943,0.208840,0.0303
3,0.032030,254174907,6800.01,NaN,3,4,3,3,1,4,...,NaN,NaN,NaN,NaN,NaN,287.258754,-41.100052,0.249032,0.231111,0.1247
4,0.050634,90935995,6801.01,NaN,3,4,4,3,1,4,...,0.3471,0.1740,0.1669,0.1826,MARCS,308.185004,-44.580820,0.273749,0.225949,0.1654


In [8]:
matches.shape

(5700, 202)

In [21]:
d = gaia_table.to_pandas().query("TOI==4364.01")
d['Gmag'], d['TESS Mag']

(3867    12.53426
 Name: Gmag, dtype: float64,
 3867    11.3993
 Name: TESS Mag, dtype: float64)

In [ ]:
d = gaia_table.to_pandas().query("TOI==4364.01")
d['Teff'], d['TESS Mag']

In [11]:
matches['GaiaDR3'] = matches['DR3Name'].apply(lambda x: int(x.split()[-1]))
matches.to_csv('../data/TOIs_GaiaDR3_xmatch.csv', index=False)

/tmp/ipykernel_3827076/3963823051.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['GaiaDR3'] = matches['DR3Name'].apply(lambda x: int(x.split()[-1]))
